## 타이타닉 예제 (머신러닝 대표 모델)
### Import

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

### CSV 파일 로딩하기

In [18]:
file_path = './titanic.csv'
df = pd.read_csv(file_path)

                 
# 데이터프레임 확인
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### 결측치 확인

In [20]:
print("shape: ",df.shape)
print("columns: ",df.columns)
print("\n=== 결측치 현황 ===")
print(df.isnull().sum())

shape:  (891, 12)
columns:  Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

=== 결측치 현황 ===
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


### 결측치 처리 (Age인 경우 평균나이로) 

In [22]:
# age 컬럼의 평균값 계산 (결측치를 제외한 값으로 계산)
mean_age = df['Age'].mean()

# age 컬럼의 결측치를 평균값으로 채우기
# fillna: 결측값을 원하는 값으로 변경하는 메서드
df['Age'] = df['Age'].fillna(mean_age)

# dropna: 결측값이 포함된 레이블을 제거하는 메서드
df = df.dropna(subset=['Embarked'])  # Embarked 결측치 제거

### 레이블 분포 확인

In [24]:
print("\n=== 레이블 분포 ===")
print(df['Survived'].value_counts())


=== 레이블 분포 ===
Survived
0    549
1    340
Name: count, dtype: int64


### 불필요한 칼럼 제거

In [26]:
cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin'] # Cabin 컬럼 제거 -> 결측치 처리 필요 X
df = df.drop(columns=cols_to_drop) # cols_to_drop 제거

### 레이블 인코딩
데이터 분할 전 인코딩을 해야함

In [28]:
# LabelEncoder 초기화: 범주형 데이터를 '정수'로 변환 
# 예) ['male', 'Female'] => [0,1]
label_encoder = LabelEncoder()

# 인코딩할 컬럼 리스트
columns_to_encode = ['Sex', 'Embarked']  # 인코딩할 컬럼 리스트

#인코딩 전 'Survived' 컬럼 값의 분포 확인
print("\n=== Survived 컬럼 인코딩 전 ===")
print(df['Survived'].value_counts())

for column in columns_to_encode:
    df[column] = label_encoder.fit_transform(df[column]) 
    # 각 범주형 데이터를 정수로 변환해 데이터 프레임에 덮어씀

#인코딩 후 'Survived' 컬럼 값의 분포 확인
print("\n=== Survived 컬럼 인코딩 후 ===")
print(df['Survived'].value_counts())


=== Survived 컬럼 인코딩 전 ===
Survived
0    549
1    340
Name: count, dtype: int64

=== Survived 컬럼 인코딩 후 ===
Survived
0    549
1    340
Name: count, dtype: int64


#### 해당 데이터셋에서는 Survived가 이미 0,1로 인코딩 되어있으므로, 인코딩 전후 차이는 없음

### 5가지 분류 (RF, DT, LR, KNN, SVM) 하고, accuracy 및 confusion matrix 확인

In [31]:
# 데이터 분할
X = df.drop('Survived', axis=1) # 특성: 데이터프레임에서 레이블(Survived)를 제외한 나머지 axis = 1 이란 열을 의미
y = df['Survived'] # 레이블
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 분류기 초기화
classifiers = {
    'Random Forest': RandomForestClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC()
}

# 모델 학습 및 평가
results = []
for name, clf in classifiers.items():
    clf.fit(X_train, y_train) # 모델 학습
    y_pred = clf.predict(X_test) # 테스트 데이터 예측
    acc = accuracy_score(y_test, y_pred) # 정확도 계산
    cm = confusion_matrix(y_test, y_pred) # 혼동 행렬 계산 (TP|FN|FP|TN)
    cr = classification_report(y_test, y_pred) # 분류 보고서
    f1 = f1_score(y_test, y_pred) # f1값 계산
    results.append((name, acc, cm, cr, f1)) # 결과를 리스트에 저장

# 결과 출력
print("\n=== 모델 성능 비교 ===")
for name, acc, cm, cr, f1 in results:
    print(f"\n======={name}=======")
    print(f"Accuracy: {acc:.4f}")
    print(f"f1_score: {f1}")
    print(cr)
    print("Confusion Matrix:")
    print(cm)


=== 모델 성능 비교 ===

=======Random Forest=======
Accuracy: 0.7584
f1_score: 0.7034482758620689
              precision    recall  f1-score   support

           0       0.82      0.77      0.80       109
           1       0.67      0.74      0.70        69

    accuracy                           0.76       178
   macro avg       0.75      0.75      0.75       178
weighted avg       0.76      0.76      0.76       178

Confusion Matrix:
[[84 25]
 [18 51]]

=======Decision Tree=======
Accuracy: 0.7528
f1_score: 0.7066666666666667
              precision    recall  f1-score   support

           0       0.84      0.74      0.79       109
           1       0.65      0.77      0.71        69

    accuracy                           0.75       178
   macro avg       0.74      0.76      0.75       178
weighted avg       0.76      0.75      0.76       178

Confusion Matrix:
[[81 28]
 [16 53]]

=======Logistic Regression=======
Accuracy: 0.7809
f1_score: 0.7346938775510204
              precision